In [ ]:
!pip install coremltools tensorflow tensorflow-addons


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.3.0
    Uninstalling typeguard-4.3.0:
      Successfully uninstalled typeguard-4.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.3.1 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [ ]:
import coremltools as ct
import tensorflow as tf
import numpy as np

# Load the Core ML model
mlmodel_path = '/content/drive/MyDrive/Data/fruithero.mlmodel'  # Replace with your .mlmodel file path
mlmodel = ct.models.MLModel(mlmodel_path)

In [ ]:
from tensorflow import keras
from tensorflow import lite


In [ ]:
import coremltools as ct
import tensorflow as tf
import numpy as np

# Load the Core ML model
mlmodel_path = '/content/drive/MyDrive/Data/fruithero.mlmodel'  # Replace with your .mlmodel file path
mlmodel = ct.models.MLModel(mlmodel_path)

# Example function to convert Core ML layers to TensorFlow Keras
def convert_coreml_to_keras(mlmodel):
    # Initialize a sequential Keras model
    keras_model = tf.keras.Sequential()

    # Example: Adding layers manually (assuming you know the architecture)
    for layer in mlmodel.get_spec().neuralNetwork.layers:
        if layer.WhichOneof('layer') == 'convolution':
            conv_layer = layer.convolution
            # Extract the weights and biases
            weights = np.array(conv_layer.weights.floatValue).reshape(
                (conv_layer.kernelChannels, conv_layer.outputChannels, conv_layer.kernelSize[0], conv_layer.kernelSize[1])
            ).transpose(2, 3, 1, 0)  # Reshape to match TensorFlow format

            biases = np.array(conv_layer.bias.floatValue)

            # Add the Conv2D layer to the Keras model
            keras_model.add(tf.keras.layers.Conv2D(
                filters=conv_layer.outputChannels,
                kernel_size=(conv_layer.kernelSize[0], conv_layer.kernelSize[1]),
                strides=(conv_layer.stride[0], conv_layer.stride[1]),
                padding='same' if conv_layer.isSamePadding else 'valid',
                weights=[weights, biases],
                activation=None
            ))

        elif layer.WhichOneof('layer') == 'activation':
            keras_model.add(tf.keras.layers.Activation('relu'))  # Example for ReLU activation

        # Add more layers handling here, such as pooling, fully connected, etc.

    # Finalize the model (assuming the output shape is known)
    keras_model.add(tf.keras.layers.Dense(units=10, activation='softmax'))  # Example output layer

    return keras_model

# Convert the Core ML model to a Keras model
tf_model = convert_coreml_to_keras(mlmodel)

# Optionally, summarize the model
tf_model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(tf_model)

In [ ]:
# Assuming 'model' is your trained Keras model
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Convert the model to TFLite format
tflite_model = converter.convert()

from google.colab import files

# Save the TFLite model to a file
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

# Download the file
files.download('model.tflite')



Saved artifact at '/tmp/tmpshf5x2k_'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 784), dtype=tf.float32, name='keras_tensor_6')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  135186114488000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135186114491520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135186114486592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135186114493104: TensorSpec(shape=(), dtype=tf.resource, name=None)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>